# Search Asset by Filter

In [ ]:
import sys
import json
import time, datetime
import pandas as pd
import requests, urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

from insightvm_authen import InsightvmAuthentication

In [ ]:
HOSTNAME = InsightvmAuthentication().getHostname()
HEADER = InsightvmAuthentication().getRequestHeader()
# HOSTNAME, HEADER

## 1. Search By Software and Tag

In [ ]:
software = "Internet Information Services 7.0"
tag="XXX"

In [ ]:
getAssetSearchURL = HOSTNAME + "api/3/assets/search"

params = {
    "size" : 100,
    "sort" : "ASC"
}

payloads = {
    "filters": [
        {
            "field": "software",
            "operator": "contains",
            "value": str(software)
        },
        {
            "field": "custom-tag",
            "operator": "contains",
            "value": str(tag)
        }
    ],
    "match": "all"
}
# payloads

In [ ]:
searchAssetBySWandTag = requests.post(getAssetSearchURL, params=params, data=json.dumps(payloads), headers=HEADER, verify=False)
searchAssetBySWandTag = searchAssetBySWandTag.json()
searchAssetBySWandTag = searchAssetBySWandTag["resources"]

In [ ]:
nowTime = datetime.datetime.now().strftime("(%y%m%d-%H%M%S) ")
filepath="outputs/" + nowTime + "asset_" + "software&tag.json"

In [ ]:
with open(filepath, 'w', encoding="utf8") as f:
    f.write(json.dumps(searchAssetBySWandTag, indent=4))
print("\n[Success Exporting] " + filepath)

## 2. Search By Scan Time Period

In [ ]:
def validateDatetime(date_text=""):
    try:
        datetime.datetime.strptime(date_text, '%Y-%m-%d')
        return date_text
    except ValueError:
        raise ValueError("Incorrect data format, should be YYYY-MM-DD")

In [ ]:
def searchAssetByScanPeriod(startDate="", stopDate=""):
    page, totalPages, totalAssets = 0, 1, 0
    datas = []
    print("\nSearch Asset ID by Asset IP")
    while page < totalPages:
        getAssetSearchURL = HOSTNAME + "api/3/assets/search?size=100&sort=ASC&page=" + str(page)
        print("Hostname : " + getAssetSearchURL)
        
        payloads = {
            "filters": [
                {
                    "field": "last-scan-date",
                    "operator": "is-between",
                    "lower": validateDatetime(date_text=startDate),
                    "upper": validateDatetime(date_text=stopDate),
                }
            ],
            "match": "all"
        }
        # payloads

        r = requests.post(getAssetSearchURL, data=json.dumps(payloads), headers=HEADER, verify=False)
        if r.status_code == 200:
            r = r.json()
            if page == 0:
                totalPages = r["page"]["totalPages"]
                print("Total Pages : " + str(totalPages))
                totalAssets = r["page"]["totalResources"]
                print("Total Assets : " + str(totalAssets))

            for line in r["resources"]:
                del(line["links"])
                datas.append(line)
            print("[Writing] Page %d of %d" % ((page + 1), (totalPages)))
            page += 1
            time.sleep(2)
        else:
            print("[Error] Request Connection Error.")
            sys.exit()
    if len(datas) == totalAssets:
        print("[Successful] %d of %d" % (len(datas), (totalAssets)))
        return datas
    else:
        print("[Error] %d of %d" % (len(datas), (totalAssets + 1)))

In [ ]:
startDate = "2021-06-01"
stopDate = "2021-08-31"

In [ ]:
searchAssetByPeriod = searchAssetByScanPeriod(startDate=startDate, stopDate=stopDate)

In [ ]:
# len(searchAssetByPeriod), searchAssetByPeriod[0]

In [ ]:
nowTime = datetime.datetime.now().strftime("(%y%m%d-%H%M%S) ")
filepath="outputs/" + nowTime + "asset-scantime_" + startDate + "-" + stopDate + ".json"

In [ ]:
with open(filepath, 'w', encoding="utf8") as f:
    f.write(json.dumps(searchAssetByPeriod, indent=4))
print("\n[Success Exporting] " + filepath)

In [ ]:
def generateHistoryString(data=[]):
#     text = []
#     for h in data:
#         linetext = [str(x) for x in h.values() if h["type"] == "SCAN"]
#         if linetext != []:
#             text.append('#'.join(linetext))
#     return ','.join(text)

    linetext = [str(x) for x in data[0].values()]
    return '#'.join(linetext)

In [ ]:
def extractData(data={}):
    return {
        "id": data["id"],
        "ip": data["ip"],
        "history": generateHistoryString(data["history"])
    }

In [ ]:
ips = [extractData(data=dat) for dat in searchAssetByPeriod]
ips = [i for i in ips if i["history"] != ""]
ips = pd.DataFrame(ips)
ips.head()

In [ ]:
nowTime = datetime.datetime.now().strftime("(%y%m%d-%H%M%S) ")
filepath="outputs/" + nowTime + "asset-scantime_" + startDate + "-" + stopDate + ".csv"

In [ ]:
ips.to_csv(filepath, sep=',', encoding='utf-8', index=False)

# Search VM Center

In [ ]:
software_search = "vCenter"

In [ ]:
getAssetSearchURL = HOSTNAME + "api/3/assets/search"

params = {
    "size" : 100,
    "sort" : "ASC"
}

payloads = {
    "filters": [
        {
            "field": "software",
            "operator": "contains",
            "value": str(software_search)
        }
    ],
    "match": "all"
}

# payloads

In [ ]:
searchAssetBySW = requests.post(getAssetSearchURL, params=params, data=json.dumps(payloads), headers=HEADER, verify=False)
searchAssetBySW = searchAssetBySW.json()
#searchAssetBySW = searchAssetBySW["resources"]

In [ ]:
def findVcenter(data=[]):
    for d in data:
        if "vcenter" in d["product"].lower():
            return {
                "software": d["product"],
               "version": d["version"]
            }
    return None

In [ ]:
def extractVcenter(data={}):
    return {**{
        "ip": data["ip"],
        "hostName": data["hostName"]
    }, **findVcenter(data["software"])}

In [ ]:
searchAssetBySW = [extractVcenter(d) for d in searchAssetBySW["resources"]]

In [ ]:
searchAssetBySW_df = pd.DataFrame(searchAssetBySW) #[["ip","hostName"]]

In [ ]:
nowTime = datetime.datetime.now().strftime("(%y%m%d-%H%M%S) ")
filepath="outputs/" + nowTime + "asset_vCenter.csv"

In [ ]:
searchAssetBySW_df.to_csv(filepath, index=False, encoding='utf-8')

# Search by CVE with Asset Tag

In [ ]:
assetTag = "<Custom Tag>"

In [ ]:
cve_text = "CVE-2021-26897"

In [ ]:
getAssetSearchURL = HOSTNAME + "api/3/assets/search"

params = {
    "size" : 100,
    "sort" : "ASC"
}

payloads = {
    "filters": [
        {
            "field": "custom-tag", #owner-tag
            "operator": "is",
            "value": str(assetTag)
        },
        {
            "field": "cve",
            "operator": "is",
            "value": str(cve_text)
        }
    ],
    "match": "all"
}

# payloads

In [ ]:
searchAssetByCVEwithAsstTag = requests.post(getAssetSearchURL, params=params, data=json.dumps(payloads), headers=HEADER, verify=False)
searchAssetByCVEwithAsstTag = searchAssetByCVEwithAsstTag.json()
searchAssetByCVEwithAsstTag["page"]

In [ ]:
searchAssetByCVEwithAsstTag = searchAssetByCVEwithAsstTag["resources"]
searchAssetByCVEwithAsstTag

Run Multiple

In [ ]:
def extractAssetInfo(data={}, cve_text=""):
     return { 
         "cve" : cve_text ,
         "ip" : data["ip"],
         "hostName" : data["hostName"],
         "softwares" : data["software"]
     }

In [ ]:
def searchAssetbyCVEWithAssetTag(assetTag="", cve_text=""):
    getAssetSearchURL = HOSTNAME + "api/3/assets/search"

    params = {
        "size" : 100,
        "sort" : "ASC"
    }

    payloads = {
        "filters": [
            {
                "field": "custom-tag",
                "operator": "is",
                "value": str(assetTag)
            },
            {
                "field": "cve",
                "operator": "is",
                "value": str(cve_text)
            }
        ],
        "match": "all"
    }

    # payloads
    searchAssetByCVEwithAsstTag = requests.post(getAssetSearchURL, params=params, data=json.dumps(payloads), headers=HEADER, verify=False)
    searchAssetByCVEwithAsstTag = searchAssetByCVEwithAsstTag.json()

    return str(searchAssetByCVEwithAsstTag["page"]["totalResources"]), [extractAssetInfo(data=asst, cve_text=cve_text) for asst in searchAssetByCVEwithAsstTag["resources"]]

In [ ]:
cves = [
    'CVE-2021-40438',
    'CVE-2021-40444',
    'CVE-2021-40449'
]

In [ ]:
assetTag = "<Custom Tag>"

assets = []

In [ ]:
for i in range(0, len(cves)): # 
    totalResources, assets_search = searchAssetbyCVEWithAssetTag(assetTag=assetTag, cve_text=cves[i])
    assets += assets_search
    print(str(i) + "," + cves[i] + "," + totalResources)
    time.sleep(10)

In [ ]:
# list(assets_search[0].keys())

In [ ]:
assets_df = pd.DataFrame(assets)
assets_df

In [ ]:
nowTime = datetime.datetime.now().strftime("(%y%m%d-%H%M%S) ")
filepath="outputs/" + nowTime + "filename.csv"

In [ ]:
assets_df.to_csv(filepath, index=False, encoding='utf-8')